![cao](https://www.cao.ie/icons/fblogo.png)

# The Central Appplications Office (CAO)


[Official Website](https://www.cao.ie/)
***

<br>

## How to load CAO points information into pandas frame

***

<br>

## Comparison of CAO points in 2019, 2020 and 2021

***

<br>

# References:

***

1.

2.

3.



***

# End